In [1]:
import pandas as pd
import numpy as np

In [2]:
#from numpy import array, delete, linalg, kron, dot, ones
from numpy import *
from math import *
from random import *

#### Code from Barbara ---- her version of simulation, so I can get characters to do Neighbor joining.

def evolveQuartet_BinaryChar_PERM(rootChar, m1, m2,m3,m4,m5,m6):  
    #start at i1 (the root),
    #go by m1 to i2, and by m2 to i3, 
    #from i2 go by m3 to e1, and by m4 to e2
    #from i3 go by m5 to e3, and by m6 to e4
    #(e for external, i for internal, m for edge mutation matrix - rows sum to 1)

    #i1 to i2 by m1
    r = random()
    row = m1[rootChar]
    if r < row[0]:
        i2 = 0
    else:
        i2 = 1

    #i1 to i3 by m2
    r = random()
    row = m2[rootChar]
    if r < row[0]:
        i3 = 0
    else:
        i3 = 1

    #i2 to e1 by m3
    r = random()
    row = m3[i2]
    if r < row[0]:
        e1 = 0
    else:
        e1 = 1

    #i2 to e2 by m4
    r = random()
    row = m4[i2]
    if r < row[0]:
        e2 = 0
    else:
        e2 = 1

    #i3 to e3 by m5
    r = random()
    row = m5[i3]
    if r < row[0]:
        e3 = 0
    else:
        e3 = 1

    #i3 to e4 by m6
    r = random()
    row = m6[i3]
    if r < row[0]:
        e4 = 0
    else:
        e4 = 1
        
    #return str(e1) + str(e3) + str(e2) + str(e4), str(e1) + str(e2) + str(e3) + str(e4), str(e1) + str(e4) + str(e3) + str(e2)
    return str(e1) + str(e2) + str(e3) + str(e4), str(e1) + str(e3) + str(e2) + str(e4), str(e1) + str(e4) + str(e3) + str(e2)
    


##############################################

def evolveQuartet_BinarySeqs_PERM(numChars,rootDist, m1,m2,m3,m4,m5,m6):   
    charsID = []
    chars23swap = []
    chars24swap = []
    for i in range(numChars):
        r = random()
        if r < rootDist[0]:
            rootChar=0
        else:
            rootChar=1
        char1, char2, char3 = evolveQuartet_BinaryChar_PERM(rootChar,m1,m2,m3,m4,m5,m6)
        charsID.append(char1)
        chars23swap.append(char2)
        chars24swap.append(char3)

    return charsID, chars23swap, chars24swap



#########################################################

def chars2PatternVector(chars):
    #there are 2^4=16 possible patterns
    #take first seq as 1s, 2nd as 4s, 3rd as 16s and 4th as 64s
    value={}
    value['0']=0
    value['1']=1

    #init pattern vector
    cv = []
    pv = []
    for i in range(16):
        cv.append(0)
        pv.append(0)
        
    #find index for each char
    for char in chars:
        patternIndex = 0
        patternIndex = patternIndex + 1*value[char[0]]
        patternIndex = patternIndex + 2*value[char[1]]
        patternIndex = patternIndex + 4*value[char[2]]
        patternIndex = patternIndex + 8*value[char[3]]
        cv[patternIndex] = cv[patternIndex] + 1

    #normalise the pv
    for i in range(16):
        pv[i] = cv[i]/float(len(chars))
        
    return cv, pv


###################################################

def flatten(p,quartet):
    hashp = {}
    if quartet == '12|34':
        hashp[0] = (0,0)
        hashp[1] = (0,1)
        hashp[2] = (0,2)
        hashp[3] = (0,3)
        hashp[4] = (1,0)
        hashp[5] = (1,1)
        hashp[6] = (1,2)
        hashp[7] = (1,3)
        hashp[8] = (2,0)
        hashp[9] = (2,1)
        hashp[10] = (2,2)
        hashp[11] = (2,3)
        hashp[12] = (3,0)
        hashp[13] = (3,1)
        hashp[14] = (3,2)
        hashp[15] = (3,3)
    elif quartet == '13|24':
        hashp[0] = (0,0)
        hashp[1] = (0,1)
        hashp[2] = (1,0)
        hashp[3] = (1,1)
        hashp[4] = (0,2)
        hashp[5] = (0,3)
        hashp[6] = (1,2)
        hashp[7] = (1,3)
        hashp[8] = (2,0)
        hashp[9] = (2,1)
        hashp[10] = (3,0)
        hashp[11] = (3,1)
        hashp[12] = (2,2)
        hashp[13] = (2,3)
        hashp[14] = (3,2)
        hashp[15] = (3,3)
    else:
        hashp[0] = (0,0)
        hashp[1] = (1,0)
        hashp[2] = (0,1)
        hashp[3] = (1,1)
        hashp[4] = (0,2)
        hashp[5] = (1,2)
        hashp[6] = (0,3)
        hashp[7] = (1,3)
        hashp[8] = (2,0)
        hashp[9] = (3,0)
        hashp[10] = (2,1)
        hashp[11] = (3,1)
        hashp[12] = (2,2)
        hashp[13] = (3,2)
        hashp[14] = (2,3)
        hashp[15] = (3,3)
    
    x = array([0.0]*16)
    flat = x.reshape((4,4))
    for i in range(16):
        flat[hashp[i][0], hashp[i][1]] = p[i]

    return flat
##########################################################

def minor(f,i,j):
    #f is a 4x4 matrix arising from 1 of 3 possible flattenings
    #i, j are indices of row and column to remove before computing determinant
    temp = delete(f, (i), axis=0)
    temp = delete(temp, (j), axis=1)
    return linalg.det(temp)

#########################################################

def allMinors(f):
    x = array([0.0]*16)
    minors = x.reshape((4,4))
    for i in range(4):
        for j in range(4):
            minors[i,j] = minor(f,i,j)
    return minors
            
#############################################################

def sumSquares(m):
    ss = 0
    for row in m:
        for cell in row:
            ss += cell*cell
    return ss

#############################################################

def RSSterm(a, b, c, sign):
    #in each case b-c = +/- a    the sign of a doesn't matter
    #E[a] = 0, E[b] = E[c] = +/-u  the sign of u is given as part of the input
    uhat = (b+c)/2
    if uhat*sign > 0: #estimate of u is the correct sign
        rss = 0.5*a*a
    else:               #estimate of u is set to 0 instead
        rss = b*b + c*c
    return rss

##########################################################
def minorRSS(m12, m13, m14):
    #input the minors from each flattening

    #algebraic constraints
    #give each one as a tuple that gives the 12 index, the 13 index, the 14 index, and the sign of u
    constraints = [([0,0], [0,0], [0,0], 1),
                   ([0,1], [0,1], [1,0], 1),
                   ([1,0], [0,2], [0,2], -1),
                   ([1,1], [0,3], [1,2], -1),
                   ([0,2], [1,0], [0,1], 1),
                   ([0,3], [1,1], [1,1], 1),
                   ([1,2], [1,2], [0,3], -1),
                   ([1,3], [1,3], [1,3], -1),
                   ([2,0], [2,0], [2,0], -1),
                   ([2,1], [2,1], [3,0], -1),
                   ([3,0], [2,2], [2,2], 1),
                   ([3,1], [2,3], [3,2], 1),
                   ([2,2], [3,0], [2,1], -1),
                   ([2,3], [3,1], [3,1], -1),
                   ([3,2], [3,2], [2,3], 1),
                   ([3,3], [3,3], [3,3], 1),]

    RSS = 0
    for cons in constraints:
        RSS += RSSterm(m12[cons[0][0], cons[0][1]], m13[cons[1][0], cons[1][1]], m14[cons[2][0], cons[2][1]], cons[3])
    return RSS

################################################################

def newerTree(sq2,sq3): #only consider 2 residuals as only 2 are independent
    #note that sq1 = -(sq2+sq3)
    u1 = (sq3-sq2)
    u2 = (-sq2-2*sq3) #sq1-sq3
    u3 = (2*sq2+sq3)    #sq2-sq1
   
    if (u1<0):  #then we set u1 to 0, ie ft the star tree
        rss1 = sq2*sq2 + sq3*sq3
    else:
        rss1 = 0.5*(sq3+sq2)*(sq3+sq2) 
    
    if (u2<0):  #then we effectively set u2 to 0, ie ft the star tree
        rss2 = sq3*sq3 + (sq3+sq2)*(sq3+sq2) 
    else:
        rss2 = 0.5*sq2*sq2
    
    if (u3<0):  #then we effectively set u1 to 0
        rss3 = sq2*sq2 + (sq3+sq2)*(sq3+sq2) 
    else:
        rss3 = 0.5*sq3*sq3
    
    return rss1, rss2, rss3

#########################################################
def chooseT1(rss1, rss2, rss3):
    if (rss1 < rss2) and (rss1 < rss3):
        return 1
    #add some random tie-breaking
    elif (rss1 == rss2) and (rss1 == rss3):
        if random() < 1/3.0:
            return 1
    elif (rss1 == rss2) and (rss1 < rss3):
        if random() < 0.5:
            return 1
    elif (rss1 == rss3) and (rss1 < rss2):
        if random() < 0.5:
            return 1 
    return 0
################################################

def chars2dists(chars):
    D = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    for char in chars:
        if char[0] != char[1]:
            D[0][1] += 1
        if char[0] != char[2]:
            D[0][2] += 1
        if char[0] != char[3]:
            D[0][3] += 1
        if char[1] != char[2]:
            D[1][2] += 1
        if char[1] != char[3]:
            D[1][3] += 1
        if char[2] != char[3]:
            D[2][3] += 1

    n = len(chars)
    D[1][0] = D[0][1] = D[0][1]/float(n) 
    D[2][0] = D[0][2] = D[0][2]/float(n)
    D[3][0] = D[0][3] = D[0][3]/float(n)
    D[2][1] = D[1][2] = D[1][2]/float(n)
    D[3][1] = D[1][3] = D[1][3]/float(n)
    D[3][2] = D[2][3] = D[2][3]/float(n)

    return D

##################################################
def correctDist(d):
    if d < 0.5:
        return -0.5*log(1 - 2*d)
    else:
        #print 'Warning dist too large to correct, just returned 3' #a bit bigger than largest observable distance in seq len of 400
        return 3

##################################################
def correctDists(D):
    D[1][0] = D[0][1] = correctDist(D[0][1])
    D[2][0] = D[0][2] = correctDist(D[0][2])
    D[3][0] = D[0][3] = correctDist(D[0][3])
    D[2][1] = D[1][2] = correctDist(D[1][2])
    D[3][1] = D[1][3] = correctDist(D[1][3])
    D[3][2] = D[2][3] = correctDist(D[2][3])

    return D
#################################################

def check4pc(D):
    #nj is equivalent to 4pc on 4 taxa
    #just check if tree 1 is correct
    small = D[0][1] + D[2][3]
    big1 = D[0][2] + D[1][3]
    big2 = D[0][3] + D[1][2]

    if (small < big1) and (small < big2):
        return 1
    #add some random tie-breaking
    elif (small == big1) and (small == big2):
        if random() < 1/3.0:
            return 1
    elif (small == big1) and (small < big2):
        if random() < 0.5:
            return 1
    elif (small == big2) and (small < big1):
        if random() < 0.5:
            return 1 
    return 0

##################################################

def unbiasedS(pv, UBSQ):
    S = 0
    for term in UBSQ:
        n = len(term)
        temp = term[0]
        for i in range(1,n):
            temp = temp*pv[term[i]]
        S += temp

    return S

######################################
#bias corrected version of decision rule
def chooseT1_BC(sq2,sq3, S1, S2, S3): #only consider 2 squangles as only 2 are independent
    #note that sq1 = -(sq2+sq3)
    u1 = (sq3-sq2)
    u2 = (-sq2-2*sq3) #sq1-sq3
    u3 = (2*sq2+sq3)    #sq2-sq1

    #if u1 is negative then we will definitely pick tree 2 or 3
    if u1 < 0:
        return 0
    #if u1 is the only positive one then we will definitely pick tree 1
    if (u2 < 0) and (u3 < 0):
        return 1

    #check for 3-way tie
    if (u1 == 0) and (u2 == 0) and (u3 == 0):
        if random() < 1/3.0:
            return 1
        return 0
    #if u1 and u2 are positive we will pick tree 1 if S1 is less than S2
    if (u2 >= 0):
        if S1 < S2:
            return 1
        elif S1 == S2: #random tie-breaking
            if random() < 0.5:
                return 1
            return 0
        else:
            return 0

    #if u1 and u3 are positive we will pick tree 1 if S1 is less than S3
    if (u3 >= 0):
        if S1 < S3:
            return 1
        elif S1 == S3:
            if random() < 0.5: #random tie-breaking
                return 1
            return 0
        else:
            return 0   

#############################################
def ericSVD(fIDby12, fIDby13, fIDby14):

    u,s12,v = linalg.svd(fIDby12)
    u,s13,v = linalg.svd(fIDby13)
    u,s14,v = linalg.svd(fIDby14)

    #I think I should use squares, should check this
    #Do I need to worry about taking the square root?
    svd_t12 = sqrt(s12[2]*s12[2] + s12[3]*s12[3])
    svd_t13 = sqrt(s13[2]*s13[2] + s13[3]*s13[3])
    svd_t14 = sqrt(s14[2]*s14[2] + s14[3]*s14[3])

    return svd_t12, svd_t13, svd_t14

#############################################
def normByRowSum(mat):
    rowSums = sum(mat, axis=1, dtype=float)

    new = array([0.0]*16).reshape((4,4))
    for i in range(4):
        if rowSums[i] == 0:
            pass
        else:
            for j in range(4):
                new[i,j] = mat[i,j]/rowSums[i]
            
    return new

############################################

def normByColSum(mat):
    colSums = sum(mat, axis=0, dtype=float)

    new = array([0.0]*16).reshape((4,4))
    for j in range(4):
        if colSums[j] == 0:
            pass
        else:
            for i in range(4):
                new[i,j] = mat[i,j]/colSums[j]
            
    return new

############################################


def ericSVD2(fIDby12, fIDby13, fIDby14):
    #normmalised version of SVD method
    #normalise by row sum
    fIDby12_rn = normByRowSum(fIDby12) 
    fIDby13_rn = normByRowSum(fIDby13)
    fIDby14_rn = normByRowSum(fIDby14) 
    #normalise by col sum
    fIDby12_cn = normByColSum(fIDby12) 
    fIDby13_cn = normByColSum(fIDby13)  
    fIDby14_cn = normByColSum(fIDby14) 
    
    u,s12r,v = linalg.svd(fIDby12_rn)
    u,s13r,v = linalg.svd(fIDby13_rn)
    u,s14r,v = linalg.svd(fIDby14_rn)

    u,s12c,v = linalg.svd(fIDby12_cn)
    u,s13c,v = linalg.svd(fIDby13_cn)
    u,s14c,v = linalg.svd(fIDby14_cn)

    #I think I should use squares, should check this
    #Do I need to worry about taking the square root?
    #also haven't bothered to divde by 2
    svd_t12 = sqrt(s12r[2]*s12r[2] + s12r[3]*s12r[3]) + sqrt(s12c[2]*s12c[2] + s12c[3]*s12c[3])
    svd_t13 = sqrt(s13r[2]*s13r[2] + s13r[3]*s13r[3]) + sqrt(s13c[2]*s13c[2] + s13c[3]*s13c[3])
    svd_t14 = sqrt(s14r[2]*s14r[2] + s14r[3]*s14r[3]) + sqrt(s14c[2]*s14c[2] + s14c[3]*s14c[3])

    return svd_t12, svd_t13, svd_t14

In [9]:
#sim parameters
Ms = [[0.95,0.05],[0.05,0.95]]
Ml = [[0.7,0.3],[0.3,0.7]]
Mmid = [[0.85,0.15],[0.15,0.85]]
Mmid2 = [[0.9,0.1],[0.1,0.9]]
#M = [[0.8,0.2],[0.2,0.8]]
Id = [[1,0],[0,1]]
rootDist = [0.4, 0.6]
REPS = 1000
#set up some matrices for transforming basis for the squangles
h = array([[1,-1],[1,1]])
ht = array([[1,1],[-1,1]])
H = kron(h,h)
Ht = kron(ht,ht)

In [10]:
# Order of input is M1, M2, M3, M4, M5, M6
# Since code identifies root in middle of an edge, and we put it at 
# an internal node, the first input and the second combine to make the 
# internal edge and we take ID as the second matrix to put the root at an internal node. 

In [11]:
dict_results = {}
for seqlen in [5*i for i in range(2,21)]:
    correctCountuNJ = 0
    correctCountNJ = 0
    
    for rep in range(REPS):
        #evolve some binary characters  
        charsID, chars23swap, chars24swap = evolveQuartet_BinarySeqs_PERM(seqlen,rootDist, Mmid2, Id, Mmid2, Mmid2, Mmid2, Mmid2) #balanced Mid
        #charsID, chars23swap, chars24swap = evolveQuartet_BinarySeqs_PERM(seqlen,rootDist, Ms, Id, Mmid, Mmid, Mmid, Mmid) #balanced Mid
        #charsID, chars23swap, chars24swap = evolveQuartet_BinarySeqs_PERM(seqlen,rootDist, Ms, Id, Ml, Ml, Ml, Ml) #balanced
        #charsID, chars23swap, chars24swap = evolveQuartet_BinarySeqs_PERM(seqlen,rootDist, Ms, Id, Ml, Ms, Ml, Ms) #Fels
        #charsID, chars23swap, chars24swap = evolveQuartet_BinarySeqs_PERM(seqlen,rootDist, Ms, Id, Ml, Ml, Ms, Ms) #Farris
        #charsID, chars23swap, chars24swap = evolveQuartet_BinarySeqs_PERM(seqlen,rootDist, Id, Id, Ml, Ml, Ms, Ms) #Star Farris

        #and good ol' NJ
        dists = chars2dists(charsID)
        correctCountuNJ += check4pc(dists)
        #print dists
        dists = correctDists(dists)
        correctCountNJ += check4pc(dists)
    
        ######end rep loop
    dict_results[seqlen] = correctCountNJ
    #summarise for this seq len
    print seqlen,  correctCountNJ
#df = pd.DataFrame.from_dict(dict_results, index = np.arange(19), columns = {'seqlen','NJ corrected'})

df = pd.DataFrame(list(dict_results.iteritems()),
                      columns=['sequence length','NJ Corrected'])
df = df.sort('sequence length', ascending = True)

10 600
15 652
20 736
25 768
30 801
35 828
40 848
45 882
50 878
55 918
60 915
65 936
70 948
75 939
80 954
85 957
90 965
95 965
100 981


/Users/ataylor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [12]:
df.to_csv('NJBalanceMid2.csv', index = False)

In [64]:
[5*i for i in range(2,41)]

[10,
 15,
 20,
 25,
 30,
 35,
 40,
 45,
 50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 110,
 115,
 120,
 125,
 130,
 135,
 140,
 145,
 150,
 155,
 160,
 165,
 170,
 175,
 180,
 185,
 190,
 195,
 200]